# Identifying Electric Vehicle Charging on Residential Power Grid

## Objective:

The city of Ipswich, MA uses electric meters which records usage patterns every hour in kilowatt hours (kWh).  Determine which residences are likely to be charging electric vehicles (EVs) based on usage patterns.  This data could be used to modify behavior through incentives to shift charging to off-peak times and reduce peak demand on the grid.

## Data:

- Initially 23 electric meters will be analyzed.  This could eventually be expanded up to identify EVs for the approximately 7000 residential meters.
- The original 23 meters were chosen because some of them are known to have EVs.  These will not be revealed until after the data is summarized.  Other houses in the list do not have confirmed EVs, but they have also not been confirmed to not have an EV.
- The data is in CSV files, where the file name is the meter number.  
- Each CSV file has at minimum the following columns: 
    - "Reading Date"
    - "Consumption (kWh)"
- Houses with solar panels will also have:
    - "Energy Received (kWh)"
- Note: Meter IDs have been modified for this project.

## Methodology:

- The accompanying PowerPoint slides discuss the trigger criteria, but the following key indicators were determined:
    - Count of usage which is 4.5 kWh higher than the average usage 2 hours prior and 2 hours after each record examined.
        - This is likely to count a high power (24 amp, 5.7 kW) EV charger while ignoring things like electric stoves, ovens, and dryers.
    - Count of usage which is at least 20 kWh over 5 hours.
        - This will catch lower power chargers based on a large usage over a long duration.
    - Total-off peak (between 12am and 7am)
        - This is intended to identity users with high overnight charging as a catch all for everything else when there is less noise in the data.  High ranking houses in this category which are not in the other two categories may be considered as potential EV houses.
        
    

- Focus on 2019 rather than 2020: Due to pandemic, EVs may not charge regularly if owner is working from home.
- Ignore summer months: Air conditioning will skew results
- Ignore winter months: Electric heat and heat pumps may skew results

Therefore, data will summarize April, May, September, and October of 2019

It is expected that there will be:
- False positives caused by other high energy uses
- False negatives caused by short duration charging, an EV with a low capacity battery (example plug in hybrids), using work or other chargers, or a low power charger.
- The intent is to try to be more inclusive with the final summary since suspected EV houses can be confirmed once the filtered list of houses is generated.



## Load Required Modules

In [29]:
import pandas as pd
import numpy as np
import datetime
import os

## Get List of File Names

In [30]:
#create list of file names
dir_name='Meter Data/' #reversed slashes to make path valid
filenamelist = os.listdir(dir_name) 
filenamelist

#merge file name and file path
pathlist=[]
for filename in filenamelist:
    pathlist.append(os.path.join(dir_name, filename))
    


## Function to Prevent Division by Zero

In [31]:
#Create function to prevent division by zero when finding average.
def safe_div(n,d):
    if d == 0:
        return 0
    return (n/d).round(1)


## Collect Data

#### Function: function_one_hour
- Compare each record to the average of 2 hours back and 2 hours forward.  Trigger if current record is 4.5 kWh greater than before and after average.

- Return:
    - Count of 1 hour triggers
    - Average value that caused trigger

In [32]:
def function_one_hour(df): 
    df['TriggerTest']=df["Consumption (kWh)"]-(df['Consumption (kWh)'].shift(2)+df['Consumption (kWh)'].shift(-2))/2
    
    df['One_Hour_Trigger']=df.apply(lambda x: 1 if x['TriggerTest'] >= 4.5 else 0, axis=1)
      
    mask_one_hr=((df['One_Hour_Trigger']-df['One_Hour_Trigger'].shift(1)) == 1)
    df=df.loc[mask_one_hr]

    trigger_avg=safe_div(df['TriggerTest'].sum(),df['TriggerTest'].count()) #take average using safe_div function to avoid dividing by zero
    trigger_count=df['One_Hour_Trigger'].count() 
    return trigger_count, trigger_avg

#### Function: function_five_hour
- Find sections of data with a usage equal to or over 20 kW for 5 hours which is an average of at least 4kWh
- Return:
    - Count of 5 hour triggers 
    - Average value that caused the trigger

In [33]:
def function_five_hour(df):
    df['Five_Hour_Sum']=df["Consumption (kWh)"]+df['Consumption (kWh)'].shift(2)+df['Consumption (kWh)'].shift(1)+df['Consumption (kWh)'].shift(-1)+df['Consumption (kWh)'].shift(-2)
    #display(df)
    df['Five_Hour_Trigger']=df.apply(lambda x: 1 if x['Five_Hour_Sum'] >= 20 else 0, axis=1)
    #display(df)
    mask_five_hr=((df['Five_Hour_Trigger']-df['Five_Hour_Trigger'].shift(1)) == 1)
    df=df.loc[mask_five_hr]
    #display(df)
    five_hour_trigger_avg=safe_div(df['Five_Hour_Sum'].sum(),df['Five_Hour_Sum'].count()) #take average using safe_div function to avoid dividing by zero
    five_hour_trigger_count=df['Five_Hour_Trigger'].count() 
    return five_hour_trigger_count, five_hour_trigger_avg

#### Function: function_night_usage

- Sum up usage data between 12AM and 6AM
- Find max peak usage between 12AM and 6AM

In [34]:
def function_night_usage(df):
    df=df.sort_values(by='Reading Date')
    df=df.set_index('Reading Date')
    df=df.between_time('01:00', '06:00') #shifted forward an hour since data is recorderded at the start of following hour
    off_peak_usage=int((df['Consumption (kWh)']).sum())
    off_peak_usage_max=int((df['Consumption (kWh)']).max())
    return off_peak_usage, off_peak_usage_max

#### Function: getvalues

- Determine if house has solar panels
- Filter by 2019
    - Collect usage summary for 2019
- Filter by April, May, September, and October
    - Collect usage summary for spring and fall
    - Collect 1hr and 5hr triggers
    - Collect off-peak summary data
- Append dataframe with all summary values for specific meter file being read

In [35]:
finallist=[]
dfworking=[]
j = 0
def getvalues(filename, year):
    global j
    dfworking=pd.read_csv(pathlist[j]) #read meter to be analyized
    j=j+1
    
    dfworking['Reading Date']=pd.to_datetime(dfworking['Reading Date']) #ensure "Reading Date" column is in date format
    meter = filename.replace('.csv', "", 1) #remove .csv suffix
    
    #Find houses with solar panels by looking for "Energy Received (kWh)" column
    if 'Energy Received (kWh)' in dfworking.columns: 
        if (dfworking['Energy Received (kWh)']).count() > 0:
            solar_panels = True
        else: solar_panels = False
    else: solar_panels = False
    
    #Create mask to filter by year
    maskinitial=((dfworking['Reading Date'] >= year+'-01-01 00:00:00') & (dfworking['Reading Date'] < str(int(year)+1)+'-01-01 00:00:00'))
    dfworking=dfworking.loc[maskinitial]     
    null_values_2019=int(dfworking['Consumption (kWh)'].isnull().sum())
    dfworking = dfworking.dropna()    
   
    #if dfworking has records
    if len(dfworking.index) > 0:    
       
        #Get max and min dates in 2019
        date_max=max(dfworking['Reading Date'])
        date_min=min(dfworking['Reading Date'])
        consumption_max_2019=int(dfworking["Consumption (kWh)"].max())

        #filter consumption over 50kWh so it data in error doesn't skew average
        maskhigh=dfworking['Consumption (kWh)'] <50
        dfworking=dfworking.loc[maskhigh]
        consumption_sum_2019=int(dfworking["Consumption (kWh)"].sum())

        #Mask to only include: Apr, May, Sep, Oct
        mask=(((dfworking['Reading Date'] >= year+'-04-01 00:00:00') & (dfworking['Reading Date'] < year+'-06-01 00:00:00')) |
        ((dfworking['Reading Date'] >= year+'-09-01 00:00:00') & (dfworking['Reading Date'] < year+'-11-01 00:00:00'))) #create mask by desired date range
        dfworking=dfworking.loc[mask]

        #if dfworking has records
        if len(dfworking.index) > 0:   
            
            #summarize data
            consumption_max=int(dfworking["Consumption (kWh)"].max())
            consumption_sum=int(dfworking["Consumption (kWh)"].sum())
            trigger_count, trigger_avg=function_one_hour(dfworking)
            five_hour_trigger_count, five_hour_trigger_avg=function_five_hour(dfworking)
            off_peak_usage, off_peak_usage_max =function_night_usage(dfworking) 
        else:
            consumption_max=0
            consumption_sum=0
            trigger_avg=0
            trigger_count=0
            five_hour_trigger_count=0
            five_hour_trigger_avg=0
            off_peak_usage=0
            off_peak_usage_max=0
    else:
        date_max=0
        date_min=0
        consumption_max_2019=0
        consumption_sum_2019=0
        consumption_max=0
        consumption_sum=0
        trigger_avg=0
        trigger_count=0
        five_hour_trigger_count=0
        five_hour_trigger_avg=0
        off_peak_usage=0
        off_peak_usage_max=0

    #print status of script
    print(j,' of ', len(filenamelist),'  |   Meter: ', meter, '|Solar: ', solar_panels,'|  1hr Trigger Count: ', trigger_count, '|  5hr Trigger Count: ', five_hour_trigger_count, '|  Consumption Sum: ', consumption_sum)
    
    #append summary of each file to "finallist"
    finallist.append([meter, solar_panels, trigger_count, trigger_avg, five_hour_trigger_count, five_hour_trigger_avg, off_peak_usage, off_peak_usage_max, consumption_max, consumption_sum, consumption_sum_2019, consumption_max_2019, null_values_2019, date_max, date_min])    
    
    return(finallist)



#### Loop through all data files available.  Use getvalues function to combine all summary data into a single dataframe.

In [36]:
#define year to be analyized
year='2019'

for i in filenamelist:
    getvalues(i, year)

#rename columns and write to new dataframe "dfsummary"
dfsummary=[]
dfsummary=pd.DataFrame(finallist, columns=["Meter Number", "Solar Panels?", "1hr Trigger Count", "1hr Trigger Average", '5hr Trigger Count', '5hr Trigger Avg', 'Offpeak Total (kwh)', 'Offpeak Max (kwh)', 'Consumption Max (kWh)', 'Consumption Sum (kWh)', 'Consumption Sum 2019 (kWh)','Consumption Max 2019 (kWh)', 'Missing Data 2019 (Hours)', 'Max Date 2019', 'Min Date 2019'])


1  of  23   |   Meter:  R02657 |Solar:  False |  1hr Trigger Count:  1 |  5hr Trigger Count:  0 |  Consumption Sum:  214
2  of  23   |   Meter:  R15467 |Solar:  False |  1hr Trigger Count:  2 |  5hr Trigger Count:  0 |  Consumption Sum:  1362
3  of  23   |   Meter:  R23573 |Solar:  False |  1hr Trigger Count:  0 |  5hr Trigger Count:  0 |  Consumption Sum:  213
4  of  23   |   Meter:  R24362 |Solar:  False |  1hr Trigger Count:  9 |  5hr Trigger Count:  10 |  Consumption Sum:  4008
5  of  23   |   Meter:  R27853 |Solar:  False |  1hr Trigger Count:  0 |  5hr Trigger Count:  0 |  Consumption Sum:  738
6  of  23   |   Meter:  R45234 |Solar:  False |  1hr Trigger Count:  0 |  5hr Trigger Count:  2 |  Consumption Sum:  3764
7  of  23   |   Meter:  R45544 |Solar:  True |  1hr Trigger Count:  9 |  5hr Trigger Count:  0 |  Consumption Sum:  1595
8  of  23   |   Meter:  R45645 |Solar:  False |  1hr Trigger Count:  0 |  5hr Trigger Count:  6 |  Consumption Sum:  4410
9  of  23   |   Meter:  R45

#### Sort final dataframe and write to CSV file.

In [37]:
#Sort data
dfsummary=dfsummary.sort_values(by=['1hr Trigger Count','5hr Trigger Count','Offpeak Total (kwh)'], ascending=False)
dfsummary=dfsummary.set_index('Meter Number')
display(dfsummary)

#write data to "FinalSummary.csv"
dfsummary.to_csv('FinalSummary'+year+'.csv')

,Solar Panels?,1hr Trigger Count,1hr Trigger Average,5hr Trigger Count,5hr Trigger Avg,Offpeak Total (kwh),Offpeak Max (kwh),Consumption Max (kWh),Consumption Sum (kWh),Consumption Sum 2019 (kWh),Consumption Max 2019 (kWh),Missing Data 2019 (Hours),Max Date 2019,Min Date 2019
Meter Number,,,,,,,,,,,,,,
R45822,False,66,4.7,75,21.1,1676,8,11,6261,27314,14,27,2019-12-31 23:00:00,2019-01-01 00:00:00
R58678,True,63,6.3,9,21.2,738,8,9,3038,9070,8668,44,2019-12-31 23:00:00,2019-01-01 00:00:00
R47564,True,26,5.9,21,23.2,590,9,13,2805,9970,13,27,2019-12-31 23:00:00,2019-01-01 00:00:00
R24362,False,9,4.9,10,20.9,579,4,8,4008,13864,9,35,2019-12-31 23:00:00,2019-01-01 00:00:00
R45544,True,9,6.1,0,0.0,515,4,9,1595,11768,15,59,2019-12-31 23:00:00,2019-01-01 00:00:00
R85461,True,7,4.7,6,21.0,978,6,7,3459,15725,375,0,2019-12-31 23:00:00,2019-01-01 00:00:00
R73486,False,5,4.8,1,20.0,331,3,7,2401,9109,8,0,2019-12-31 23:00:00,2019-01-01 00:00:00
R48911,True,2,6.2,1,27.0,500,9,9,1516,6075,9,22,2019-06-06 23:00:00,2019-01-01 00:00:00
R75310,False,2,4.5,1,20.0,398,1,6,1854,7506,9,0,2019-12-31 23:00:00,2019-01-01 00:00:00


### Conclusions

After summarizing and reviewing data as well as adding in the known EVs, the data was color coded by an apparent drop off in the number of triggers.  While this part is open to interpretation, the idea was to look for a clear drop-off which could be adjusted if necessary, to control how many false positives were acceptable.
The initial criteria are:
- 6x or higher 1 hour triggers
- 6x or higher 5 hour triggers
- 500 kWh total usage & at least 4 kWh Max Peak during off-peak hours

If a house has at least 2 of the above criteria it is consider a high probability EV house.
If it only has 1 of the criteria it is considered a low probability EV house.

#### Below is the summary table after manipulating in Excel to add extra information and to color code
<img src="Town Data Final Summary-No Yearly Summary.PNG" />

Based on the above criteria the final tally is below.  Keep in mind that the criteria could be adjusted after knowing which houses have an EV.  The idea is there is likely always going to be some false positives and false negatives, but the criteria must be selected to minimize them.

- 12 houses were categorized as to not have an EV
- 5 houses were categorized to have an EV and have a confirmed EV.
- 4 houses were false positives, however upon visual inspection all 4 off them had some EV charging qualities, so they are considered good false positives to include in the count since they may turn out to have EVs.
- 2 houses with false negatives, however upon visual inspection both of these did not show EV charging qualities.  These owners may have an EV with a small battery or utilize off site charging.

#### Overall this trial run of the data was deemed successful and is ready to expand to a much larger dataset to identify EV owners and reach out to them for potential rate plan incentives to get them to not charge during periods of peak usage.  

Future improvements:
- The color coding of high and medium probability was done by hand since it was unclear what a good cutoff point would be.  On a larger dataset the rules should be programed in to automatically identify each group based on their "trigger count".
- Have a larger dataset of confirmed houses with and without EVs.  Where not all the houses had this information confirmed, it was difficult to make rules based on assumptions.




